In [1139]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [1140]:
data = pd.read_csv("dataset.csv", na_values = "?")

# Data Preparation (Categorical Data)

In [1141]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    90 non-null     int64  
 1   Date            90 non-null     object 
 2   BoosterVersion  90 non-null     object 
 3   PayloadMass     90 non-null     float64
 4   Orbit           90 non-null     object 
 5   LaunchSite      90 non-null     object 
 6   Outcome         90 non-null     object 
 7   Flights         90 non-null     int64  
 8   GridFins        90 non-null     bool   
 9   Reused          90 non-null     bool   
 10  Legs            90 non-null     bool   
 11  LandingPad      64 non-null     object 
 12  Block           90 non-null     float64
 13  ReusedCount     90 non-null     int64  
 14  Serial          90 non-null     object 
 15  Longitude       90 non-null     float64
 16  Latitude        90 non-null     float64
 17  Class           90 non-null     int64

## Find Categorical Data

### Data Format Correction

In [1142]:
data["Class"] = data["Class"].astype("bool")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    90 non-null     int64  
 1   Date            90 non-null     object 
 2   BoosterVersion  90 non-null     object 
 3   PayloadMass     90 non-null     float64
 4   Orbit           90 non-null     object 
 5   LaunchSite      90 non-null     object 
 6   Outcome         90 non-null     object 
 7   Flights         90 non-null     int64  
 8   GridFins        90 non-null     bool   
 9   Reused          90 non-null     bool   
 10  Legs            90 non-null     bool   
 11  LandingPad      64 non-null     object 
 12  Block           90 non-null     float64
 13  ReusedCount     90 non-null     int64  
 14  Serial          90 non-null     object 
 15  Longitude       90 non-null     float64
 16  Latitude        90 non-null     float64
 17  Class           90 non-null     bool 

In [1143]:
cat_data = data.select_dtypes(include = {object, bool})
cat_data.head()

,Date,BoosterVersion,Orbit,LaunchSite,Outcome,GridFins,Reused,Legs,LandingPad,Serial,Class
0,2010-06-04,Falcon 9,LEO,CCAFS SLC 40,None None,False,False,False,NaN,B0003,False
1,2012-05-22,Falcon 9,LEO,CCAFS SLC 40,None None,False,False,False,NaN,B0005,False
2,2013-03-01,Falcon 9,ISS,CCAFS SLC 40,None None,False,False,False,NaN,B0007,False
3,2013-09-29,Falcon 9,PO,VAFB SLC 4E,False Ocean,False,False,False,NaN,B1003,False
4,2013-12-03,Falcon 9,GTO,CCAFS SLC 40,None None,False,False,False,NaN,B1004,False


## Selecting Data

In [1144]:
cat_data.nunique()

Date              90
BoosterVersion     1
Orbit             11
LaunchSite         3
Outcome            8
GridFins           2
Reused             2
Legs               2
LandingPad         5
Serial            53
Class              2
dtype: int64

### Select Low Variance Data

Since the BoosterVersion feature has only one value, we decided to drop the BoosterVersion feature which is not relevant to our data mining goal

In [1172]:
cat_data.drop("BoosterVersion", axis = 1, inplace = True)
cat_data.head(5)

,Date,Orbit,LaunchSite,Outcome,GridFins,Reused,Legs,LandingPad,Serial,Class
0,2010-06-04,LEO,CCAFS SLC 40,None None,False,False,False,NaN,B0003,False
1,2012-05-22,LEO,CCAFS SLC 40,None None,False,False,False,NaN,B0005,False
2,2013-03-01,ISS,CCAFS SLC 40,None None,False,False,False,NaN,B0007,False
3,2013-09-29,PO,VAFB SLC 4E,False Ocean,False,False,False,NaN,B1003,False
4,2013-12-03,GTO,CCAFS SLC 40,None None,False,False,False,NaN,B1004,False


## Split Data

Split data into training and testing data set.

In [1146]:
# Load the python library of sklearn
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing data set
train, test = train_test_split(cat_data, test_size = 0.2, random_state = 2)

print('Original Dataset Shape: ', cat_data.shape)
print('Training Dataset Shape: ', train.shape)
print('Testing Dataset Shape:', test.shape)

Original Dataset Shape:  (90, 10)
Training Dataset Shape:  (72, 10)
Testing Dataset Shape: (18, 10)


## Cleaning Data (Training Dataset)

### Missing Value In Training Dataset

Identify and calculate the percentage of the missing value of each feature in training dataset

In [1147]:
train.isnull().sum()/len(train)*100

Date           0.000000
Orbit          0.000000
LaunchSite     0.000000
Outcome        0.000000
GridFins       0.000000
Reused         0.000000
Legs           0.000000
LandingPad    31.944444
Serial         0.000000
Class          0.000000
dtype: float64

The result shows the percentage of the missing value of the LandingPad feature in the training dataset is 31.94% which is lower than 80%. Therefore, we are not consider to drop the whole column or all records with missing value but decided to replace the missing value with the most frequent value.

In [1148]:
# cat_data.mode().loc[0,"LandingPad"]
new_cat_data = train.fillna(train.mode().iloc[0,7])
new_cat_data.head(5).sort_index()

,Date,Orbit,LaunchSite,Outcome,GridFins,Reused,Legs,LandingPad,Serial,Class
27,2017-03-16,GTO,KSC LC 39A,None None,False,False,False,5e9e3032383ecb6bb234e7ca,B1030,False
30,2017-05-15,GTO,KSC LC 39A,None None,False,False,False,5e9e3032383ecb6bb234e7ca,B1034,False
35,2017-08-14,ISS,KSC LC 39A,True RTLS,True,False,True,5e9e3032383ecb267a34e7c7,B1039,True
44,2018-01-31,GTO,CCAFS SLC 40,True Ocean,True,True,True,5e9e3032383ecb6bb234e7ca,B1032,True
57,2018-11-15,GTO,KSC LC 39A,True ASDS,True,True,True,5e9e3032383ecb6bb234e7ca,B1047,True


In [1149]:
new_cat_data.isnull().sum()

Date          0
Orbit         0
LaunchSite    0
Outcome       0
GridFins      0
Reused        0
Legs          0
LandingPad    0
Serial        0
Class         0
dtype: int64

### Outliers

No outlier detection for categorical data

### Duplicate Data

In [1150]:
new_cat_data.duplicated().sum()

0

There is no duplicate record in the training dataset

## Handling Categorical Attributes (Training Dataset)

Before handling the categorical attributes, the number of category of each feature is identified.

In [1151]:
# Number of different values per category
orbit_cat = pd.get_dummies(new_cat_data["Orbit"]).shape
launch_site_cat = pd.get_dummies(new_cat_data["LaunchSite"]).shape
landing_pad_cat = pd.get_dummies(new_cat_data["LandingPad"]).shape
serial_cat = pd.get_dummies(new_cat_data["Serial"]).shape

print("Number of category in Orbit: %s"%(orbit_cat[1]))
print("Number of category in LaunchSite: %s"%(launch_site_cat[1]))
print("Number of category in LandingPad: %s"%(landing_pad_cat[1]))
print("Number of category in Serial: %s"%(serial_cat[1]))

Number of category in Orbit: 10
Number of category in LaunchSite: 3
Number of category in LandingPad: 4
Number of category in Serial: 45


Based on the result, we can see that the feature 'Serial' in the training set contains 45 different values which will increase 45 columns to the data set after applied the OneHotEncoder. Consider the low amount of records in the data set, the number of features should be limited to avoid the problem of overfitting. Therefore, the feature 'Serial' will not be applied the OneHotDecoder and considered to be dropped due to it hold less amount of decision compared to other features as said in the Categorical Feature Analysis section.

Generate dummies variable to apply OneHotEncoder to the categorical columns Orbits, LaunchSite, and LandingPad.

In [1152]:
columns = ["Orbit", "LaunchSite", "LandingPad"]
dummies = pd.get_dummies(new_cat_data[columns])
output = pd.concat([new_cat_data.drop(columns = columns), dummies], axis = 1)
output.head(5).sort_index()

,Date,Outcome,GridFins,Reused,Legs,Serial,Class,Orbit_ES-L1,Orbit_GEO,Orbit_GTO,...,Orbit_SO,Orbit_SSO,Orbit_VLEO,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb554034e7c9,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc
27,2017-03-16,None None,False,False,False,B1030,False,0,0,1,...,0,0,0,0,1,0,0,0,1,0
30,2017-05-15,None None,False,False,False,B1034,False,0,0,1,...,0,0,0,0,1,0,0,0,1,0
35,2017-08-14,True RTLS,True,False,True,B1039,True,0,0,0,...,0,0,0,0,1,0,1,0,0,0
44,2018-01-31,True Ocean,True,True,True,B1032,True,0,0,1,...,0,0,0,1,0,0,0,0,1,0
57,2018-11-15,True ASDS,True,True,True,B1047,True,0,0,1,...,0,0,0,0,1,0,0,0,1,0


In [1153]:
print("Number of categorical features before applied OneHotEncoder: %s"%(new_cat_data.shape[1]))
print("Number of categorical features after applied OneHotEncoder: %s"%(output.shape[1]))

Number of categorical features before applied OneHotEncoder: 10
Number of categorical features after applied OneHotEncoder: 24


## Feature Selection (Training Dataset)

### Feature Subset Selection

Identify and calculate the pairwise correlation of each feature pair. Identify and consider whether to drop the feature subset with high pairwise correlation or keep it.

In [1154]:
new_cat_data.corr()

,GridFins,Reused,Legs,Class
GridFins,1.000000,0.219240,0.882778,0.647496
Reused,0.219240,1.000000,0.127442,0.198341
Legs,0.882778,0.127442,1.000000,0.685061
Class,0.647496,0.198341,0.685061,1.000000


From above result, it shows the feature 'GridFins' and 'Legs' has a very high correlation of 0.88 which is larger than the value of 0.80 to consider very high correlation between two attributes. However, based on the feature definition, there is a low correlation between the two features. Therefore, we decided to not drop the features.

### Construct Data

Consider feature 'Date' is not a relevant feature to predict the future landing outcome of the SpaceX rocket. New feature of 'Month' are decided to be constructed by extracting feature from the feature 'Date' as consider it is a useful feature to the prediction of y.

In [1155]:
date = output.loc[:, "Date"]
flight_month = []

for i in date:
    flight_month.append(i.split("-")[1])
flight_month_df = pd.DataFrame(flight_month, columns = ["Month"])

output["Date"] = flight_month_df
output.rename(columns = {'Date':'Month'}, inplace = True)
output.head(5).sort_index()

,Month,Outcome,GridFins,Reused,Legs,Serial,Class,Orbit_ES-L1,Orbit_GEO,Orbit_GTO,...,Orbit_SO,Orbit_SSO,Orbit_VLEO,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb554034e7c9,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc
27,12,None None,False,False,False,B1030,False,0,0,1,...,0,0,0,0,1,0,0,0,1,0
30,09,None None,False,False,False,B1034,False,0,0,1,...,0,0,0,0,1,0,0,0,1,0
35,01,True RTLS,True,False,True,B1039,True,0,0,0,...,0,0,0,0,1,0,1,0,0,0
44,12,True Ocean,True,True,True,B1032,True,0,0,1,...,0,0,0,1,0,0,0,0,1,0
57,05,True ASDS,True,True,True,B1047,True,0,0,1,...,0,0,0,0,1,0,0,0,1,0


### Remove Feature

Drop the redundant feature 'Outcome' due to it is highly correlated to the y variable 'Class' by it contains True to indicate y = 1 and None or False to indicate y = 0. Also, it contains another information to indicate the landing region but the information is considered useless for our data mining goal to only predict whether the rocket can successfully landed or not.

In [1156]:
output.drop("Outcome", axis = 1, inplace = True)
output.head(5)

,Month,GridFins,Reused,Legs,Serial,Class,Orbit_ES-L1,Orbit_GEO,Orbit_GTO,Orbit_ISS,...,Orbit_SO,Orbit_SSO,Orbit_VLEO,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb554034e7c9,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc
30,09,False,False,False,B1034,False,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
27,12,False,False,False,B1030,False,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
44,12,True,True,True,B1032,True,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
35,01,True,False,True,B1039,True,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
57,05,True,True,True,B1047,True,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


Drop the feature 'Serial' due to it contains too many number of category which is not suitable to apply OneHotEncoder in this case and the data is very diverse to contribute less to the y prediction.

In [1157]:
output.drop("Serial", axis = 1, inplace = True)
output.head(5)

,Month,GridFins,Reused,Legs,Class,Orbit_ES-L1,Orbit_GEO,Orbit_GTO,Orbit_ISS,Orbit_LEO,...,Orbit_SO,Orbit_SSO,Orbit_VLEO,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb554034e7c9,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc
30,09,False,False,False,False,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
27,12,False,False,False,False,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
44,12,True,True,True,True,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
35,01,True,False,True,True,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
57,05,True,True,True,True,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


## Feature Scaling

No feature scaling for categorical data

## Cleaning Data (Testing Dataset)

### Missing Value In Testing Dataset

Identify and calculate the percentage of the missing value of each feature in training dataset

In [1158]:
test.isnull().sum()/len(test)*100

Date           0.000000
Orbit          0.000000
LaunchSite     0.000000
Outcome        0.000000
GridFins       0.000000
Reused         0.000000
Legs           0.000000
LandingPad    16.666667
Serial         0.000000
Class          0.000000
dtype: float64

The same method to replace missing value in the training dataset will be applied to the testing dataset. There is 16.67% of missing values in the feature 'LandingPad' in the testing dataset and will be replaced by the most frequent value in the same column of training dataset, which is the value of '5e9e3032383ecb6bb234e7ca'

In [1159]:
new_test_data = test.fillna(train.mode().iloc[0,7])
new_test_data.head(5).sort_index()

,Date,Orbit,LaunchSite,Outcome,GridFins,Reused,Legs,LandingPad,Serial,Class
36,2017-08-24,SSO,VAFB SLC 4E,True ASDS,True,False,True,5e9e3033383ecbb9e534e7cc,B1038,True
48,2018-04-18,HEO,CCAFS SLC 40,True ASDS,True,False,True,5e9e3032383ecb6bb234e7ca,B1045,True
77,2020-04-22,VLEO,KSC LC 39A,True ASDS,True,True,True,5e9e3032383ecb6bb234e7ca,B1051,True
79,2020-06-04,VLEO,CCAFS SLC 40,True ASDS,True,True,True,5e9e3033383ecbb9e534e7cc,B1049,True
81,2020-06-30,MEO,CCAFS SLC 40,True ASDS,True,False,True,5e9e3033383ecbb9e534e7cc,B1060,True


In [1160]:
new_test_data.isnull().sum()

Date          0
Orbit         0
LaunchSite    0
Outcome       0
GridFins      0
Reused        0
Legs          0
LandingPad    0
Serial        0
Class         0
dtype: int64

### Duplicate Data

In [1161]:
new_test_data.duplicated().sum()

0

There is no duplicate record in the testing dataset.

## Handling Categorical Attributes (Testing Dataset)

In [1162]:
# Number of different values per category
orbit_cat = pd.get_dummies(new_test_data["Orbit"]).shape
launch_site_cat = pd.get_dummies(new_test_data["LaunchSite"]).shape
landing_pad_cat = pd.get_dummies(new_test_data["LandingPad"]).shape
serial_cat = pd.get_dummies(new_test_data["Serial"]).shape

print("Number of category in Orbit: %s"%(orbit_cat[1]))
print("Number of category in LaunchSite: %s"%(launch_site_cat[1]))
print("Number of category in LandingPad: %s"%(landing_pad_cat[1]))
print("Number of category in Serial: %s"%(serial_cat[1]))

Number of category in Orbit: 8
Number of category in LaunchSite: 3
Number of category in LandingPad: 4
Number of category in Serial: 16


The feature 'Serial' will not be applied the OneHotDecoder as same as in the training dataset. Generate dummies variable to apply OneHotEncoder to the categorical columns Orbits, LaunchSite, and LandingPad.

In [1163]:
columns = ["Orbit", "LaunchSite", "LandingPad"]
dummies = pd.get_dummies(new_test_data[columns])
test_output = pd.concat([new_test_data.drop(columns = columns), dummies], axis = 1)
test_output.head(5).sort_index()

,Date,Outcome,GridFins,Reused,Legs,Serial,Class,Orbit_GTO,Orbit_HEO,Orbit_ISS,...,Orbit_PO,Orbit_SSO,Orbit_VLEO,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3032383ecb761634e7cb,LandingPad_5e9e3033383ecbb9e534e7cc
36,2017-08-24,True ASDS,True,False,True,B1038,True,0,0,0,...,0,1,0,0,0,1,0,0,0,1
48,2018-04-18,True ASDS,True,False,True,B1045,True,0,1,0,...,0,0,0,1,0,0,0,1,0,0
77,2020-04-22,True ASDS,True,True,True,B1051,True,0,0,0,...,0,0,1,0,1,0,0,1,0,0
79,2020-06-04,True ASDS,True,True,True,B1049,True,0,0,0,...,0,0,1,1,0,0,0,0,0,1
81,2020-06-30,True ASDS,True,False,True,B1060,True,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [1164]:
print("Number of categorical features before applied OneHotEncoder: %s"%(new_test_data.shape[1]))
print("Number of categorical features after applied OneHotEncoder: %s"%(test_output.shape[1]))

Number of categorical features before applied OneHotEncoder: 10
Number of categorical features after applied OneHotEncoder: 22


Compare the features those had applied OneHotDecoder in the training dataset and the testing dataset and ensure the testing dataset has the same features with the training dataset

In [1165]:
# Drop the features those are in the testing dataset but not in the training data set

train_column = output.iloc[:, 5:].columns
test_column = test_output.iloc[:, 7:].columns

drop_column = []
for i in test_column:
    if not (i in train_column):
        drop_column.append(i)
test_output.drop(columns = drop_column, axis = 1, inplace = True)
test_output.iloc[:, 7:].shape

(18, 13)

In [1166]:
# Add the features those are in the training dataset but not in the testing dataset, replace the missing values with 0

train_column = output.iloc[:, 5:].columns
test_column = test_output.iloc[:, 7:].columns

add_column = []
for i in train_column:
    if not (i in test_column):
        add_column.append(i)
for j in add_column:
    test_output["%s"%j] = 0
test_output.iloc[:, 7:].shape

(18, 17)

In [1167]:
# Compare is the training dataset and the testing dataset contain the same features after applied the OneHotEncoder
train_test = pd.concat([output.iloc[:, 5:], test_output.iloc[:, 7:]], axis = 0)
train_test.shape

(90, 17)

In [1168]:
print("Number of categorical features before applied OneHotEncoder: %s"%(new_test_data.shape[1]))
print("Number of categorical features after applied OneHotEncoder: %s"%(test_output.shape[1]))

Number of categorical features before applied OneHotEncoder: 10
Number of categorical features after applied OneHotEncoder: 24


## Feature Selection (Testing Dataset)

In [1169]:
# Drop the features those had dropped in the training dataset
# Drop the feature 'Date' and add new feature of 'Month' extracted from Date

test_date = test_output.loc[:, "Date"]
test_flight_month = []

for i in test_date:
    test_flight_month.append(i.split("-")[1])
test_flight_month_df = pd.DataFrame(test_flight_month)

test_output.rename(columns = {'Date':'Month'}, inplace = True)
test_output["Month"] = test_flight_month_df.values
test_output.head(5).sort_index()

,Month,Outcome,GridFins,Reused,Legs,Serial,Class,Orbit_GTO,Orbit_ISS,Orbit_LEO,...,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc,Orbit_ES-L1,Orbit_GEO,Orbit_SO,LandingPad_5e9e3032383ecb554034e7c9
36,08,True ASDS,True,False,True,B1038,True,0,0,0,...,0,0,1,0,0,1,0,0,0,0
48,04,True ASDS,True,False,True,B1045,True,0,0,0,...,1,0,0,0,1,0,0,0,0,0
77,04,True ASDS,True,True,True,B1051,True,0,0,0,...,0,1,0,0,1,0,0,0,0,0
79,06,True ASDS,True,True,True,B1049,True,0,0,0,...,1,0,0,0,0,1,0,0,0,0
81,06,True ASDS,True,False,True,B1060,True,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [1170]:
# Drop the feature 'Outcome' and 'Serial'

test_output.drop("Outcome", axis = 1, inplace = True)
test_output.drop("Serial", axis = 1, inplace = True)
test_output.head(5)

,Month,GridFins,Reused,Legs,Class,Orbit_GTO,Orbit_ISS,Orbit_LEO,Orbit_MEO,Orbit_PO,...,LaunchSite_CCAFS SLC 40,LaunchSite_KSC LC 39A,LaunchSite_VAFB SLC 4E,LandingPad_5e9e3032383ecb267a34e7c7,LandingPad_5e9e3032383ecb6bb234e7ca,LandingPad_5e9e3033383ecbb9e534e7cc,Orbit_ES-L1,Orbit_GEO,Orbit_SO,LandingPad_5e9e3032383ecb554034e7c9
48,04,True,False,True,True,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
77,04,True,True,True,True,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
81,06,True,False,True,True,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
79,06,True,True,True,True,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
36,08,True,False,True,True,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


## Finalize Training Dataset And Testing Dataset

In [1171]:
cat_training_dataset = output
cat_testing_dataset = test_output

print("Shape of training dataset: {}".format(cat_training_dataset.shape))
print("Shape of testing dataset: {}".format(cat_testing_dataset.shape))

Shape of training dataset: (72, 22)
Shape of testing dataset: (18, 22)
